<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv">link</a>.

**NOTE**:

For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01"><img src="https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [1]:
%load_ext sql

In [2]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://smt18684:Lsb8ouTylZNDetDW@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb?security=SSL

'Connected: smt18684@bludb'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [12]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT TABSCHEMA, TABNAME, CREATE_TIME FROM SYSCAT.TABLES WHERE TABSCHEMA = 'smt18684';
# %sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='smt18684'
# %sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[]

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [10]:
# type in your query to retrieve the number of columns in the SCHOOLS table
# %sql SELECT count(*) from SCHOOLS;
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(78,)]

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [19]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT COLNAME, TYPENAME, LENGTH from syscat.columns WHERE tabname = 'SCHOOLS';

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('SCHOOL_ID', 'INTEGER', 4),
 ('NAME_OF_SCHOOL', 'VARCHAR', 64),
 ('Elementary, Middle, or High School', 'VARCHAR', 2),
 ('STREET_ADDRESS', 'VARCHAR', 29),
 ('CITY', 'VARCHAR', 7),
 ('STATE', 'VARCHAR', 2),
 ('ZIP_CODE', 'INTEGER', 4),
 ('PHONE_NUMBER', 'VARCHAR', 14),
 ('LINK', 'VARCHAR', 78),
 ('NETWORK_MANAGER', 'VARCHAR', 40),
 ('COLLABORATIVE_NAME', 'VARCHAR', 34),
 ('ADEQUATE_YEARLY_PROGRESS_MADE_', 'VARCHAR', 3),
 ('TRACK_SCHEDULE', 'VARCHAR', 12),
 ('CPS_PERFORMANCE_POLICY_STATUS', 'VARCHAR', 16),
 ('CPS_PERFORMANCE_POLICY_LEVEL', 'VARCHAR', 15),
 ('HEALTHY_SCHOOL_CERTIFIED', 'VARCHAR', 3),
 ('SAFETY_ICON', 'VARCHAR', 11),
 ('SAFETY_SCORE', 'SMALLINT', 2),
 ('FAMILY_INVOLVEMENT_ICON', 'VARCHAR', 11),
 ('FAMILY_INVOLVEMENT_SCORE', 'VARCHAR', 3),
 ('ENVIRONMENT_ICON', 'VARCHAR', 11),
 ('ENVIRONMENT_SCORE', 'SMALLINT', 2),
 ('INSTRUCTION_ICON', 'VARCHAR', 11),
 ('INSTRUCTION_SCORE', 'SMALLINT', 2),
 ('LEADERS_ICON', 'VARCHAR', 4),
 ('LEADERS_SCORE', 'VARCHAR', 3),
 ('TEACHERS_ICON', 'VARCHAR', 11),
 ('TEACHERS_SCORE', 'VARCHAR', 3),
 ('PARENT_ENGAGEMENT_ICON', 'VARCHAR', 7),
 ('PARENT_ENGAGEMENT_SCORE', 'VARCHAR', 3),
 ('PARENT_ENVIRONMENT_ICON', 'VARCHAR', 7),
 ('PARENT_ENVIRONMENT_SCORE', 'VARCHAR', 3),
 ('AVERAGE_STUDENT_ATTENDANCE', 'VARCHAR', 6),
 ('RATE_OF_MISCONDUCTS__PER_100_STUDENTS_', 'DECIMAL', 5),
 ('AVERAGE_TEACHER_ATTENDANCE', 'VARCHAR', 6),
 ('INDIVIDUALIZED_EDUCATION_PROGRAM_COMPLIANCE_RATE', 'VARCHAR', 7),
 ('PK_2_LITERACY__', 'VARCHAR', 4),
 ('PK_2_MATH__', 'VARCHAR', 4),
 ('GR3_5_GRADE_LEVEL_MATH__', 'VARCHAR', 4),
 ('GR3_5_GRADE_LEVEL_READ__', 'VARCHAR', 4),
 ('GR3_5_KEEP_PACE_READ__', 'VARCHAR', 4),
 ('GR3_5_KEEP_PACE_MATH__', 'VARCHAR', 4),
 ('GR6_8_GRADE_LEVEL_MATH__', 'VARCHAR', 4),
 ('GR6_8_GRADE_LEVEL_READ__', 'VARCHAR', 4),
 ('GR6_8_KEEP_PACE_MATH_', 'VARCHAR', 4),
 ('GR6_8_KEEP_PACE_READ__', 'VARCHAR', 4),
 ('GR_8_EXPLORE_MATH__', 'VARCHAR', 4),
 ('GR_8_EXPLORE_READ__', 'VARCHAR', 4),
 ('ISAT_EXCEEDING_MATH__', 'DECIMAL', 4),
 ('ISAT_EXCEEDING_READING__', 'DECIMAL', 4),
 ('ISAT_VALUE_ADD_MATH', 'DECIMAL', 3),
 ('ISAT_VALUE_ADD_READ', 'DECIMAL', 3),
 ('ISAT_VALUE_ADD_COLOR_MATH', 'VARCHAR', 6),
 ('ISAT_VALUE_ADD_COLOR_READ', 'VARCHAR', 6),
 ('STUDENTS_TAKING__ALGEBRA__', 'VARCHAR', 4),
 ('STUDENTS_PASSING__ALGEBRA__', 'VARCHAR', 4),
 ('9th Grade EXPLORE (2009)', 'VARCHAR', 4),
 ('9th Grade EXPLORE (2010)', 'VARCHAR', 4),
 ('10th Grade PLAN (2009)', 'VARCHAR', 4),
 ('10th Grade PLAN (2010)', 'VARCHAR', 4),
 ('NET_CHANGE_EXPLORE_AND_PLAN', 'VARCHAR', 3),
 ('11th Grade Average ACT (2011)', 'VARCHAR', 4),
 ('NET_CHANGE_PLAN_AND_ACT', 'VARCHAR', 3),
 ('COLLEGE_ELIGIBILITY__', 'VARCHAR', 4),
 ('GRADUATION_RATE__', 'VARCHAR', 4),
 ('COLLEGE_ENROLLMENT_RATE__', 'VARCHAR', 4),
 ('COLLEGE_ENROLLMENT', 'SMALLINT', 2),
 ('GENERAL_SERVICES_ROUTE', 'SMALLINT', 2),
 ('FRESHMAN_ON_TRACK_RATE__', 'VARCHAR', 4),
 ('X_COORDINATE', 'DECIMAL', 13),
 ('Y_COORDINATE', 'DECIMAL', 13),
 ('LATITUDE', 'DECIMAL', 18),
 ('LONGITUDE', 'DECIMAL', 18),
 ('COMMUNITY_AREA_NUMBER', 'SMALLINT', 2),
 ('COMMUNITY_AREA_NAME', 'VARCHAR', 22),
 ('WARD', 'SMALLINT', 2),
 ('POLICE_DISTRICT', 'SMALLINT', 2),
 ('LOCATION', 'VARCHAR', 27)]

Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [27]:
%sql SELECT COUNT(*) FROM SCHOOLS WHERE "Elementary, Middle, or High School" = 'ES' ;
# column name contains MiXeD-cAsE, which be quoted with "DOUBLE QUOTEs" 

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(462,)]

Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [41]:
%%sql 
select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: Statement Execute Failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0198N  The statement string of the PREPARE or EXECUTE IMMEDIATE statement is blank or empty.  SQLSTATE=42617 SQLCODE=-198
[SQL: -- select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS;]
(Background on this error at: http://sqlalche.me/e/13/f405)


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [44]:
%sql SELECT * FROM SCHOOLS WHERE SAFETY_SCORE = (SELECT MAX(SAFETY_SCORE) FROM SCHOOLS);

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(610038, 'Abraham Lincoln Elementary School', 'ES', '615 W Kemper Pl', 'Chicago', 'IL', 60614, '(773) 534-5720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Very Strong', 99, 'Very Strong', '99', 'Strong', 74, 'Strong', 66, 'Weak', '65', 'Strong', '70', 'Strong', '56', 'Average', '47', '96.00%', Decimal('2.0'), '96.40%', '95.80%', '80.1', '43.3', '89.6', '84.9', '60.7', '62.6', '81.9', '85.2', '52', '62.4', '66.3', '77.9', Decimal('69.7'), Decimal('64.4'), Decimal('0.2'), Decimal('0.9'), 'Yellow', 'Green', '67.1', '54.5', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 813, 33, 'NDA', Decimal('1171699.458'), Decimal('1915829.428'), Decimal('41.92449696'), Decimal('-87.64452163'), 7, 'LINCOLN PARK', 43, 18, '(41.92449696, -87.64452163)'),
 (609799, 'Alexander Graham Bell Elementary School', 'ES', '3730 N Oakley Ave', 'Chicago', 'IL', 60618, '(773) 534-5150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609799.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99, 'Very Strong', '88', 'Strong', 64, 'Average', 46, 'Weak', '51', 'Average', '51', 'NDA', 'NDA', 'NDA', 'NDA', '96.30%', Decimal('6.3'), '95.90%', '99.30%', '91.9', '67.3', '79.2', '77.4', '53.3', '54.7', '84.2', '83', '49.8', '53.6', '62.6', '71.7', Decimal('64.0'), Decimal('57.9'), Decimal('0.0'), Decimal('0.3'), 'Yellow', 'Yellow', '58.1', '65.6', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 998, 35, 'NDA', Decimal('1160327.881'), Decimal('1924862.722'), Decimal('41.94952795'), Decimal('-87.68605496'), 5, 'NORTH CENTER', 47, 19, '(41.94952795, -87.68605496)'),
 (610084, 'Annie Keller Elementary Gifted Magnet School', 'ES', '3020 W 108th St', 'Chicago', 'IL', 60655, '(773) 535-2636', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610084.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99, 'Very Strong', '97', 'Very Strong', 85, 'Very Strong', 82, 'Weak', '94', 'Very Strong', '82', 'Strong', '68', 'Strong', '60', '97.50%', Decimal('4.9'), '96.50%', '100.00%', '100', 'NDA', '100', '100', '63.4', '74.7', '96.5', '97.7', '61.2', '76.7', '89.3', '100', Decimal('92.8'), Decimal('92.3'), Decimal('1.6'), Decimal('2.3'), 'Green', 'Green', '51.5', '70.6', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 245, 49, 'NDA', Decimal('1157959.455'), Decimal('1832892.067'), Decimal('41.69719792'), Decimal('-87.69726380'), 74, 'MOUNT GREENWOOD', 19, 22, '(41.69719792, -87.6972638)'),
 (609820, 'Augustus H Burley Elementary School', 'ES', '1630 W Barry Ave', 'Chicago', 'IL', 60657, '(773) 534-5475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609820.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99, 'NDA', 'NDA', 'Strong', 78, 'Strong', 65, 'Weak', 'NDA', 'NDA', 'NDA', 'Strong', '59', 'Average', '49', '96.50%', Decimal('0.7'), '95.00%', '97.90%', '69.4', '47', '64.5', '70.2', '51.2', '66.7', '69.4', '75', '62.9', '67.3', '50', '69.6', Decimal('54.2'), Decimal('53.3'), Decimal('1.3'), Decimal('2.1'), 'Green', 'Green', '35.7', '80', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 'NDA', 572, 33, 'NDA', Decimal('1164768.164'), Decimal('1920682.831'), Decimal('41.93796493'), Decimal('-87.66985204'), 6, 'LAKE VIEW', 32, 19, '(41.93796493, -87.66985204)'),
 (610132, 'Edgar Allan Poe Elementary Classical School', 'ES', '10538 S Langley Ave', 'Chicago', 'IL', 60628, '(773) 535-5525', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610132.pdf', 'Lake Calumet Element

Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [56]:
%sql SELECT Name_of_School,Average_Student_Attendance FROM SCHOOLS ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC NULLS LAST LIMIT 10;
# %sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    # order by Average_Student_Attendance desc nulls last limit 10 

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('John Charles Haines Elementary School', '98.40%'),
 ('James Ward Elementary School', '97.80%'),
 ('Edgar Allan Poe Elementary Classical School', '97.60%'),
 ('Orozco Fine Arts & Sciences Elementary School', '97.60%'),
 ('Rachel Carson Elementary School', '97.60%'),
 ('Annie Keller Elementary Gifted Magnet School', '97.50%'),
 ('Andrew Jackson Elementary Language Academy', '97.40%'),
 ('Lenart Elementary Regional Gifted Center', '97.40%'),
 ('Disney II Magnet School', '97.30%'),
 ('John H Vanderpoel Elementary Magnet School', '97.20%')]

Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [62]:
%sql SELECT Name_of_School,Average_Student_Attendance FROM SCHOOLS ORDER BY AVERAGE_STUDENT_ATTENDANCE ASC NULLS LAST LIMIT 10;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('Richard T Crane Technical Preparatory High School', '57.90'),
 ('Barbara Vick Early Childhood & Family Center', '60.90'),
 ('Dyett High School', '62.50'),
 ('Wendell Phillips Academy High School', '63.00'),
 ('Orr Academy High School', '66.30'),
 ('Manley Career Academy High School', '66.80'),
 ('Chicago Vocational Career Academy High School', '68.80'),
 ('Roberto Clemente Community Academy High School', '69.60'),
 ('Austin Polytechnical Academy High School', '70.10'),
 ('Hyde Park Academy High School', '70.50')]

Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [ ]:
%sql SELECT Name_of_School,REPLACE (Average_Student_Attendance, '%','') FROM SCHOOLS ORDER BY AVERAGE_STUDENT_ATTENDANCE ASC NULLS LAST LIMIT 10;

Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [76]:
%%sql 
SELECT Name_of_School,Average_Student_Attendance FROM SCHOOLS WHERE DECIMAL ( REPLACE (Average_Student_Attendance, '%','') ) < 70 ORDER BY Average_Student_Attendance ;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('Richard T Crane Technical Preparatory High School', '57.90%'),
 ('Barbara Vick Early Childhood & Family Center', '60.90%'),
 ('Dyett High School', '62.50%'),
 ('Wendell Phillips Academy High School', '63.00%'),
 ('Orr Academy High School', '66.30%'),
 ('Manley Career Academy High School', '66.80%'),
 ('Chicago Vocational Career Academy High School', '68.80%'),
 ('Roberto Clemente Community Academy High School', '69.60%')]

Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [78]:
%sql SELECT SUM(college_enrollment),community_area_name FROM SCHOOLS GROUP BY community_area_name;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(6864, 'ALBANY PARK'),
 (4823, 'ARCHER HEIGHTS'),
 (1458, 'ARMOUR SQUARE'),
 (6483, 'ASHBURN'),
 (4175, 'AUBURN GRESHAM'),
 (10933, 'AUSTIN'),
 (1522, 'AVALON PARK'),
 (3640, 'AVONDALE'),
 (14386, 'BELMONT CRAGIN'),
 (1636, 'BEVERLY'),
 (3167, 'BRIDGEPORT'),
 (9647, 'BRIGHTON PARK'),
 (549, 'BURNSIDE'),
 (1568, 'CALUMET HEIGHTS'),
 (5042, 'CHATHAM'),
 (7086, 'CHICAGO LAWN'),
 (2085, 'CLEARING'),
 (4670, 'DOUGLAS'),
 (4568, 'DUNNING'),
 (5337, 'EAST GARFIELD PARK'),
 (5305, 'EAST SIDE'),
 (4600, 'EDGEWATER'),
 (910, 'EDISON PARK'),
 (6832, 'ENGLEWOOD'),
 (1431, 'FOREST GLEN'),
 (531, 'FULLER PARK'),
 (9915, 'GAGE PARK'),
 (4552, 'GARFIELD RIDGE'),
 (2809, 'GRAND BOULEVARD'),
 (4051, 'GREATER GRAND CROSSING'),
 (963, 'HEGEWISCH'),
 (3975, 'HERMOSA'),
 (8620, 'HUMBOLDT PARK'),
 (1930, 'HYDE PARK'),
 (7764, 'IRVING PARK'),
 (1755, 'JEFFERSON PARK'),
 (4287, 'KENWOOD'),
 (7055, 'LAKE VIEW'),
 (5615, 'LINCOLN PARK'),
 (4132, 'LINCOLN SQUARE'),
 (7351, 'LOGAN SQUARE'),
 (871, 'LOOP'),
 (7257, 'LOWER WEST SIDE'),
 (1552, 'MCKINLEY PARK'),
 (1317, 'MONTCLARE'),
 (3271, 'MORGAN PARK'),
 (2091, 'MOUNT GREENWOOD'),
 (3362, 'NEAR NORTH SIDE'),
 (1378, 'NEAR SOUTH SIDE'),
 (7975, 'NEAR WEST SIDE'),
 (7922, 'NEW CITY'),
 (7541, 'NORTH CENTER'),
 (5146, 'NORTH LAWNDALE'),
 (4210, 'NORTH PARK'),
 (6469, 'NORWOOD PARK'),
 (140, 'OAKLAND'),
 (786, 'OHARE'),
 (6954, 'PORTAGE PARK'),
 (1620, 'PULLMAN'),
 (1547, 'RIVERDALE'),
 (4068, 'ROGERS PARK'),
 (7020, 'ROSELAND'),
 (4043, 'SOUTH CHICAGO'),
 (1859, 'SOUTH DEERING'),
 (14793, 'SOUTH LAWNDALE'),
 (4543, 'SOUTH SHORE'),
 (4388, 'UPTOWN'),
 (4006, 'WASHINGTON HEIGHTS'),
 (2648, 'WASHINGTON PARK'),
 (3700, 'WEST ELSDON'),
 (5946, 'WEST ENGLEWOOD'),
 (2622, 'WEST GARFIELD PARK'),
 (4207, 'WEST LAWN'),
 (3240, 'WEST PULLMAN'),
 (8197, 'WEST RIDGE'),
 (9429, 'WEST TOWN'),
 (4206, 'WOODLAWN')]

Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [81]:
%sql SELECT community_area_name, SUM(college_enrollment) FROM SCHOOLS GROUP BY community_area_name ORDER BY SUM(college_enrollment) LIMIT 5;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('OAKLAND', 140),
 ('FULLER PARK', 531),
 ('BURNSIDE', 549),
 ('OHARE', 786),
 ('LOOP', 871)]

Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [83]:
%sql SELECT name_of_school, SAFETY_SCORE FROM SCHOOLS ORDER BY SAFETY_SCORE LIMIT 5;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[('Edmond Burke Elementary School', 1),
 ("Luke O'Toole Elementary School", 5),
 ('George W Tilton Elementary School', 6),
 ('Foster Park Elementary School', 11),
 ('Emil G Hirsch Metropolitan High School', 13)]

Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [86]:
%sql SELECT HARDSHIP_INDEX FROM chicago_socioeconomic_data CD, SCHOOLS SC WHERE CD.ca = SC.community_area_number and college_enrollment = 4368;

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(6.0,)]

Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [88]:
#%sql SELECT HARDSHIP_INDEX FROM chicago_socioeconomic_data CD, SCHOOLS SC WHERE CD.ca = SC.community_area_number and college_enrollment = (SELECT MAX(college_enrollment) FROM SCHOOLS);
%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

 * ibm_db_sa://smt18684:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/bludb
Done.


[(5.0, 'North Center', 6.0)]

Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2021-07-09        | 2.4     | Malika     | Updated connection string  |
| 2021-05-19        | 2.3     | Lakshmi Holla |Updated question     |
| 2021-04-20        | 2.2     | Malika     | Added the libraries                |
| 2020-11-27        | 2.1     | Sannareddy Ramesh    | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
